In [ ]:
import cv2
import glob
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [ ]:
def extract_landmarks(results):
    left_hand = np.array([[result_left.x, result_left.y, result_left.z] for result_left in results.left_hand_landmarks.landmark]) if results.left_hand_landmarks else np.zeros((21,3))
    right_hand = np.array([[result_right.x, result_right.y, result_right.z] for result_right in results.right_hand_landmarks.landmark]) if results.right_hand_landmarks else np.zeros((21,3))
    hands = np.concatenate([left_hand.flatten(), right_hand.flatten()])
    return hands

# Exemplo uma imagem

In [ ]:
image =  cv2.imread('exemplo.jpeg')   
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB

mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    image.flags.writeable = False                  # Image is no longer writeable
    results = holistic.process(image)              # Make prediction

    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=1, circle_radius=2), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=1, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=1, circle_radius=2), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=1, circle_radius=2)
                             ) 
    
#     draw_styled_landmarks(image, results)          # Hands landmarks draw
    plt.imshow(image)
    
    keypoints = extract_landmarks(results)         # extract landmarks 

# Salvar landmarks de um dataset

In [ ]:
#### banco de dados
## classe 1
# imagem 1
# imagem 2
## classe 2
# imagem 1



IMG_SIZE = (640, 480)
path = 'dataset' # dataset de entrada 
labels_ = os.listdir(path)


data_points = "dataset_landmarks" # cria um dataset de saída
if not os.path.exists(data_points):
    os.makedirs(data_points)

mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    for i, label in enumerate(labels_):
        cur_path = path + "/" + label 
        
        diretorio = os.path.join(data_points, label) 
        if not os.path.exists(diretorio):
            os.mkdir(diretorio)
       
        for image_path in glob.glob(cur_path + "/*"):
            image = cv2.imread(image_path)
            image = cv2.resize(image, IMG_SIZE)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            image.flags.writeable = False                  # Image is no longer writeable
            results = holistic.process(image)              # Make prediction

            keypoints = extract_landmarks(results)         # extract landmarks 
            npy_path = os.path.join(data_points, label, image_path.split("\\")[-1][:-4])
            np.save(npy_path, keypoints) # salvar npy

        print ("[INFO] completed label - " + label)
